In [ ]:
import os
import json
import torch
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct" #"nlpai-lab/KULLM3" #"rtzr/ko-gemma-2-9b-it"

device = 'cuda' if torch.cuda.is_available() else 'cpu'

llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True 
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm.eval()

In [ ]:
instruction1 = """
당신은 신문 기자 입니다. 이런 근데 누군가가 신문 제목을 망쳐놨군요. 당신은 신문 기자의 경험을 살려 망가진 제목을 복구해야합니다. 설명을 붙이지 말고 복구한 제목만 출력하세요.
User Input: {}
"""


In [ ]:
instruction2 = """
당신은 주어진 컨텍스트를 파악해서 original 컨텍스트가 정말로 noise가 껴있던게 맞는 것인지 파악해야합니다. 왼쪽이 오리지널이고 오른쪽이 복구된 겁니다.
설명을 붙이지 말고 'noised' 나 노이즈가 아니라면 'nanoise' 중 하나를 출력하세요.
User Input: {}
"""


In [ ]:
with open("../data/train.csv", "r") as f:
    df = pd.read_csv(f)

In [ ]:
def make_prompt(instruction: str, text: str):
    return instruction.format(text)

def find_pre_noise(text: str):
    messages = [
    {"role": "system", 
    "content": "당신에게 컨텍스트 하나가 주어질 때 노이즈가 낀 상태인지 파악해야합니다. 노이즈의 종류는 특수문자와 영어 철자입니다. 단 '...', '·', '…' 과 한자는 노이즈가 아닙니다. 특히 '만루 안타·45...' 이렇게 끝이 '...' 있는 것은 기사 제목이 너무 길어서 나타나는 자연스러운 현상이므로 노이즈가 아니다. 또한 제목이 자연스럽게 해석이 되면은 노이즈가 아니다. 설명을 붙이지 말고 노이즈가 껴있던 거라면 'noised', 노이즈가 없는거라면 'nanoise'를 출력하세요."},
    {"role": "user", "content": text}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )
    # prompt = make_prompt(instruction2, context)
    # inputs = tokenizer.encode(
    #     prompt,
    #     return_tensors='pt'
    # ).to(device)

    outputs = llm.generate(
        inputs.to(device),
        max_new_tokens=256,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(generated_text)

    # response = generated_text[len(prompt):].strip()
    # print(response)

    result = None
    result = generated_text.split("[|assistant|]")[-1].strip()
    
    # if "생성 답변:" in response:
    #     result = response.split("생성 답변:")[-1].strip()
    # elif "### 생성 답변 ###\n" in response:
    #     result = response.split("### 생성 답변 ###\n")[-1].strip()
        
    return result

def find_post_noise(text1: str, text2: str):
    context = text1 + "  " + text2
    messages = [
    {"role": "system", 
    "content": "당신에게 컨텍스트 하나가 주어질 때 왼쪽과 오른쪽의 텍스트를 비교해서 오른쪽에 비해서 왼쪽이 노이즈가 많이 낀 상태인지 파악해야합니다. 설명을 붙이지 말고 노이즈가 껴있던 거라면 'noised', 노이즈가 없는거라면 'nanoise'를 출력하세요."},
    {"role": "user", "content": context}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )
    # prompt = make_prompt(instruction2, context)
    # inputs = tokenizer.encode(
    #     prompt,
    #     return_tensors='pt'
    # ).to(device)

    outputs = llm.generate(
        inputs.to(device),
        max_new_tokens=256,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(generated_text)

    # response = generated_text[len(prompt):].strip()
    # print(response)

    result = None
    result = generated_text.split("[|assistant|]")[-1].strip()
    
    # if "생성 답변:" in response:
    #     result = response.split("생성 답변:")[-1].strip()
    # elif "### 생성 답변 ###\n" in response:
    #     result = response.split("### 생성 답변 ###\n")[-1].strip()
        
    return result

In [ ]:
def cleaning_noise(row: pd.DataFrame) -> list:
    text = row['text']

    result = find_pre_noise(text)
    if result != "noised": return None
    
    messages = [
    {"role": "system", 
     "content": "당신은 신문 기자 입니다. 이런 근데 누군가가 신문 제목을 망쳐놨군요. 당신은 신문 기자의 경험을 살려 망가진 제목을 말이되게 복구해야합니다. 설명을 붙이지 말고 복구한 제목 하나만 출력하세요."},
    {"role": "user", "content": text}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )
    # prompt = make_prompt(instruction1, text)
    # inputs = tokenizer.encode(
    #     prompt,
    #     return_tensors='pt'
    # ).to(device)

    outputs = llm.generate(
        inputs.to(device),
        max_new_tokens=256,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # response = generated_text[len(prompt):].strip()
    # print(response)
    
    restored_data = None
    restored_text = generated_text.split("[|assistant|]")[-1].strip()

    # print(restored_text)
    # result = find_post_noise(text, restored_text)

    if result == 'noised':
        restored_data = {
            'ID': row['ID'],
            'text': restored_text,
            'target': row['target']
        }
    

    # if "생성 답변:" in response:
    #     restored_text = response.split("생성 답변:")[-1].strip()
    #     result = find_noise(text, restored_text)
        
    #     if result == 'noised':
    #         restored_data = {
    #             'ID': row['ID'],
    #             'text': restored_text,
    #             'target': row['target']
    #         }
    # elif "### 생성 답변 ###\n" in response:
    #     restored_text = response.split("### 생성 답변 ###\n")[-1].strip()
    #     result = find_noise(text, restored_text)

    #     if result == 'noised':
    #         restored_data = {
    #             'ID': row['ID'],
    #             'text': restored_text,
    #             'target': row['target']
    #         }
        
    return restored_data

In [ ]:
from tqdm.notebook import tqdm 
restored_datas = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Cleaning-noise"):
    restored_result = cleaning_noise(row)
    if restored_result is None: 
        continue
    restored_datas.append(restored_result)

In [ ]:
# for data in restored_datas:
#     print(data)

In [ ]:
df = pd.DataFrame(restored_datas)
df.to_csv("../data/restored_train_data5.csv", index=False, encoding="utf-8-sig")

print(len(restored_datas))